In [ ]:
!pip install git+https://github.com/huggingface/transformers accelerate qwen-vl-utils bitsandbytes

In [ ]:
!pip install torch

In [ ]:
from datasets import Dataset

# Percorso del file salvato
# json_path = "/kaggle/input/final-test-set/final_dataset.json"
dataset_path = '/kaggle/input/summarized-dataset/dataset_summarized'

# Carica il dataset Hugging Face
# hf_dataset = Dataset.from_json(json_path)
hf_dataset = Dataset.load_from_disk(dataset_path)

# Controlla un esempio
print(hf_dataset)

In [ ]:
from transformers import pipeline

# Carica il classificatore zero-shot
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Etichette da prevedere
import re

LABELS = ["real", "fake"]

def extract_label_from_response(response: str) -> int:
    result = classifier(response, LABELS)
    return LABELS.index(result["labels"][0])

In [ ]:
from transformers import (
    LlavaNextProcessor,
    LlavaNextForConditionalGeneration,
    BitsAndBytesConfig,
    GenerationConfig
)
from PIL import Image
import torch
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# Few-shot esempi per real/fake
FEW_SHOT_EXAMPLES = [
    {
        "context": "A street photo taken at midday with perfectly natural shadows and realistic reflections on wet pavement.",
        "label": "[Real]",
        "motivation": "Consistent lighting and natural shadow falloff; reflections are coherent with the surfaces."
    },
    {
        "context": "An indoor scene where the textures on the walls appear overly smooth and uniform, and shadows lack realistic variation.",
        "label": "[Fake]",
        "motivation": "Uniform texture and flat shadows are typical AI generation artifacts."
    },
]

STATIC_PROMPTS = [
    "Is this image real or fake? Choose one: [Real] / [Fake].",
    "Analyze and classify: [Real] / [Fake].",
    "Based on lighting and texture, decide: [Real] / [Fake]."
]

class LlavaTester:
    def __init__(self, model_id="llava-hf/llava-v1.6-mistral-7b-hf", device="cuda:0"):
        self.device = torch.device(device)
        self.processor = LlavaNextProcessor.from_pretrained(model_id)

        self.model = LlavaNextForConditionalGeneration.from_pretrained(
            model_id, 
            torch_dtype=torch.float16, 
            low_cpu_mem_usage=True,
            load_in_4bit=True
        )

    def build_prompt(self, prompt_type="dynamic", x_t=""):
        few_shot = "".join(
            f"Example:\nContext: {ex['context']}\nAnswer: {ex['label']} Motivation: {ex['motivation']}\n\n"
            for ex in FEW_SHOT_EXAMPLES
        )
        if prompt_type == "dynamic" and x_t:
            return (
                "You are a classifier that labels images as [Real] or [Fake].\n"
                "Analyze the following description and the image, return ONLY one label: [Real] or [Fake]. Do not explain.\n\n"
                f"Image Description and Technical Analysis:\n{x_t.strip()}\n\n"
                "Answer with exactly one of the two options:\n[Real]\n[Fake]\n\n"
                "Answer:"
            )
        else:
            idx = int(prompt_type.split("_")[-1]) if "_" in prompt_type else 0
            template = STATIC_PROMPTS[idx]
            return few_shot + f"Question: {template}\nAnswer:"

    def generate(self, image_path: str, prompt: str) -> str:
        # 1) load+resize
        img = Image.open(image_path).convert("RGB")
        

        # 2) build chat
        chat = self.processor.apply_chat_template(
            [{"role":"user","content":[{"type":"text","text":prompt},{"type":"image"}]}],
            add_generation_prompt=True
        )

        # 3) tokenize + image
        inputs = self.processor(text=chat, images=img, return_tensors="pt", padding=True).to(self.device)

        # 4) generate
        out = self.model.generate(
            **inputs,
            max_new_tokens=100,
        )

        response = self.processor.decode(out[0], skip_special_tokens=True).strip()

        # Estrai solo la risposta finale, ignorando i few-shot examples
        # Trova l'ultima occorrenza di "Answer:" e restituisci la parte successiva
        answer_start = response.rfind("Answer:")  # Trova l'ultima occorrenza di "Answer:"
        if answer_start != -1:
            response = response[answer_start:].split("\n")[-1].strip()  # Ottieni solo la parte dopo "Answer:"
        return response

    def extract_label(self, response: str) -> int:
        return extract_label_from_response(response)

    def test(self, example: dict, prompt_type="dynamic"):
        torch.cuda.empty_cache()
        prompt = self.build_prompt(prompt_type, example.get("summarized_x_t",""))
        resp = self.generate(example["image_k"], prompt)
        label = self.extract_label(resp)
        return label, resp

# wrapper
_tester = LlavaTester()
def test_llava(example, prompt_type="dynamic"):
    return _tester.test(example, prompt_type)


In [ ]:
from datasets import load_from_disk
from tqdm import tqdm
import pandas as pd

def run_test():
    #dataset = load_from_disk(DATASET_PATH)
    for i in range(10):
        #print("Dataset structure:\n", hf_dataset)
        example = hf_dataset[i]
        print("Ground_Truth:\n", example['label'])
        #print("Image:\n", example['image'])
        #print("Prompt:\n", example['x_t'])
        label, response = test_llava(example, "dynamic")
        print("Output label:\n", label)
        print("Output response:\n", response)

#run_test()


In [ ]:
import os
import pandas as pd
from tqdm.auto import tqdm
from datasets import load_from_disk

# DATASET_PATH   = "/kaggle/input/test-dataset-kaggle/test_dataset_kaggle"
CHECKPOINT_CSV = "/kaggle/working/dynamic_new_dataset_sum_llava.csv"
PROMPT_TYPE    = "dynamic"
CHECKPOINT_EVERY = 30  # salva ogni 30 campioni

# 1) Carica dataset
#dataset = load_from_disk(DATASET_PATH)

# 2) Se esiste già un CSV di checkpoint, riloadalo e salta i campioni già processati
if os.path.exists(CHECKPOINT_CSV):
    df = pd.read_csv(CHECKPOINT_CSV)
    processed_ids = set(df["img_id"])
else:
    df = pd.DataFrame(columns=["img_id","gt_label","pred_label","response"])
    processed_ids = set()

# 3) Loop con tqdm e checkpoint ogni N campioni
for sample in tqdm(hf_dataset, desc=f"Eval {PROMPT_TYPE}", dynamic_ncols=True):
    img_id = sample["img_id"]
    if img_id in processed_ids:
        continue

    try:
        pred_label, resp = test_llava(sample, prompt_type=PROMPT_TYPE)
    except Exception as e:
        print("Errore nella generazione")
        pred_label, resp = -1, f"[ERROR] {e}"

    # prepara la nuova riga
    row = {
        "img_id":     img_id,
        "gt_label":   sample["label"],
        "pred_label": pred_label,
        "response":   resp
    }
    # concatena in un colpo solo
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    processed_ids.add(img_id)

    # salva checkpoint
    if len(processed_ids) % CHECKPOINT_EVERY == 0:
        df.to_csv(CHECKPOINT_CSV, index=False)

# 4) Alla fine salva il CSV definitivo
df.to_csv(CHECKPOINT_CSV, index=False)
print("✅ Checkpoint salvato in:", CHECKPOINT_CSV)

# 5) Calcolo accuracy su quelli validi
valid = df["pred_label"] != -1
acc = (df.loc[valid, "gt_label"] == df.loc[valid, "pred_label"]).mean()
print(f"Accuracy {PROMPT_TYPE}: {acc:.4f}")
